In [1]:
import requests
import json
import pandas as pd

In [2]:
#Get data on community areas 
r_places = requests.get('https://chicagohealth.herokuapp.com/api/v1/places')
places_data = r_places.json()

In [8]:
#Get data on all resources in chicago
r_resources = requests.get('https://chicagohealth.herokuapp.com/api/v1/resources_all')
resources_data = r_resources.json()

In [11]:
#Get 1990, 2000, 2010 data on life expectancy
r_le_1990 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/1990/life-expectancy')
r_le_2000 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2000/life-expectancy')
r_le_2010 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2010/life-expectancy')

In [12]:
le_1990_data = r_le_1990.json()
le_2000_data = r_le_2000.json()
le_2010_data = r_le_2010.json() 

In [18]:
#Convert to Pandas DF and then to CSV

geo_name, geo_id, ind_id, le = [],[],[],[]
for result in le_1990_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
le_1990 = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'life_exp': le})
le_1990.to_csv(".\data\CHA_le_1990.csv")

geo_name, geo_id, ind_id, le = [],[],[],[]
for result in le_2000_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
le_2000 = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'life_exp': le})
le_2000.to_csv(".\data\CHA_le_2000.csv")

geo_name, geo_id, ind_id, le = [],[],[],[]
for result in le_2010_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
le_2010 = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'life_exp': le})
le_2010.to_csv(".\data\CHA_le_2010.csv")

,geo_id,geo_name,ind_id,life_exp
0,224979,1-Rogers Park,652,73
1,224980,2-West Ridge,652,78
2,224981,3-Uptown,652,71
3,224982,4-Lincoln Square,652,76
4,224983,5-North Center,652,77


In [23]:
#Get 2016 overall health status data
r_ohs_2016 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2014-2016/overall-health-status')
ohs_2016_data = r_ohs_2016.json()

geo_name, geo_id, ind_id, ohs_low_ci, ohs_hi_ci, wt_num, ohs_weight = [],[],[],[],[],[], []
for result in ohs_2016_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
    ohs_low_ci.append(result["lower_95ci_weight_percent"])
    ohs_hi_ci.append(result["upper_95ci_weight_percent"])
    wt_num.append(result["weight_number"])
    ohs_weight.append(result["weight_percent"])
ohs_2016 = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'ohs_low_ci': ohs_low_ci, 'ohs_hi_ci': ohs_hi_ci, 
                       'wt_num': wt_num, 'ohs_weight': ohs_weight})
ohs_2016.to_csv(".\data\ohs_2016.csv")

,geo_id,geo_name,ind_id,ohs_hi_ci,ohs_low_ci,ohs_weight,wt_num
0,224433,1-Rogers Park,649,87.9,69.3,78.6,37000.0
1,224434,2-West Ridge,649,94.5,83.1,88.8,53600.0
2,224435,3-Uptown,649,95.7,80.7,88.2,32500.0
3,224436,4-Lincoln Square,649,94.6,81.5,88.1,29000.0
4,224437,5-North Center,649,100.0,82.2,91.9,18300.0


In [32]:
#Get 2000, 2010 infant mortality data
r_im_2000 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2000-2004/infant-mortality')
r_im_2010 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2010-2014/infant-mortality')
r_im_2000_data = r_im_2000.json()
r_im_2010_data = r_im_2010.json()